## Evaluación de un modelo y su aplicación para predicción

> **María de los Angeles Arista Huerta - A01369984**

Entregable 2 - Portafolio de analisis / *NOVIEMBRE 2022*

Importar librerias

In [ ]:
# Manipulación de los datos
import numpy as np
import pandas as pd

# Visualización de los datos
import seaborn as sns
import matplotlib.pyplot as plt

# Modelos
#import statsmodels.api as sm
#import statsmodels.formula.api as sm
#from statsmodels.tsa.stattools import adfuller

In [ ]:
# acceso a directorio
%cd "c:\Users\angix\Downloads\Estadistica"
#%ls 

In [ ]:
#Carga del dataset

df = pd.read_csv("c:\\Users\\angix\\Downloads\\Evidence2\\Indicadores.csv")
df